In [16]:
!pip install metric_learn
from datapreparation.profile_parser import *
import metric_learn
import pandas as pd
import numpy as np
from  sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

# Profile Similarity

#### Params
1. Managed content file for cleaned up profile data
2. Profile ID for the account for which we need to check similarity
3. Similarity threshold
4. PCA boolean Flag
5. Numeric/Dense column Flag

#### Response
1. Array of tuples with profile_id and similarity scores in sorted order

#### Next steps
1. Currently data distribution is assumed Gaussian
2. Does not perform well with a dense column (amount field). Need to check how to mix sparse matrix with dense column
3. Need a supervised approach to separate disparate data points
4. Need to include inferred attributes like entities, sentiments, topics and tone from Notes data (Once it is available)


In [3]:
token=''
url="https://api.cortex.insights.ai/v3/agents/c360/customer-360-toplevel/services/retrieve-profile"
final_list=parse_profile_data(url,token)
print('Parsing complete')
original_df = pd.DataFrame(final_list)
original_df['Amount1']=original_df.apply(remove_text, axis=1)

final_df=get_df_competitors(original_df)

final_df_with_dummies = pd.get_dummies( final_df, columns = ['competitor_names','ExecSponsor1','OpportunityName1','OpportunityStatus1','industry_name'] )
# final_df_with_dummies.to_csv("profiledata.csv", sep=',', index=False,encoding='utf-8')

Parsing complete


In [17]:
industries=final_df['industry_name']
le=LabelEncoder()
# print(le.fit_transform(industries))
# print(industries)

target=le.fit_transform(industries)
final_df_with_dummies['Target']=target
final_df_with_dummies.head()

,Amount1,account_name,contacts,profile_id,competitor_names_AIG,competitor_names_AT&T,competitor_names_Aetna,competitor_names_Albertsons Cos.,competitor_names_Alphabet,competitor_names_Amazon.com,...,industry_name_Network and Other Communications Equipment,industry_name_Petroleum Refining,industry_name_Pharmaceuticals,industry_name_Pipelines,industry_name_Semiconductors and Other Electronic Components,industry_name_Specialty Retailers: Other,industry_name_Telecommunications,industry_name_Wholesalers: Food and Grocery,industry_name_Wholesalers: Health Care,Target
0,50000,Walmart,[Eduardo Brayner],0015I11484m7u87wBuD,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
1,50000,Costco,[Karim Krzan],0011S65285x2u62kQjM,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
2,50000,Target,[Michael McNamara],0018K30946c0u64tLmE,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
3,10580000,Exxon Mobil,[Ridwan Zurawik],0017K23751y0u72fZpZ,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,27
4,50000,Chevron,[David Seals],0010V22883q6u92eKgN,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,27


In [18]:
final_df.head()

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
0,50000,Rosella Hymer,Amplify - QuickStart(Process Insights),Confirming Value with Power,Walmart,Costco,[Eduardo Brayner],General Merchandisers,0015I11484m7u87wBuD
1,50000,Axel Whitsel,Engage - QuickStart(Coversational AI),Prospecting,Costco,Target,[Karim Krzan],General Merchandisers,0011S65285x2u62kQjM
2,50000,Jayne Knox,Amplify - QuickStart(Process Insights),Confirming Value with Power,Target,Walmart,[Michael McNamara],General Merchandisers,0018K30946c0u64tLmE
3,10580000,Astrid Schmaus,Amplify - Enterprise(Survellance Insights),Use Case Confirmed,Exxon Mobil,Chevron,[Ridwan Zurawik],Petroleum Refining,0017K23751y0u72fZpZ
4,50000,Olly Breuninger,Amplify - QuickStart(Survellance Insights),Use Case Confirmed,Chevron,Phillips 66,[David Seals],Petroleum Refining,0010V22883q6u92eKgN


In [19]:
final_df_with_dummies.head()

,Amount1,account_name,contacts,profile_id,competitor_names_AIG,competitor_names_AT&T,competitor_names_Aetna,competitor_names_Albertsons Cos.,competitor_names_Alphabet,competitor_names_Amazon.com,...,industry_name_Network and Other Communications Equipment,industry_name_Petroleum Refining,industry_name_Pharmaceuticals,industry_name_Pipelines,industry_name_Semiconductors and Other Electronic Components,industry_name_Specialty Retailers: Other,industry_name_Telecommunications,industry_name_Wholesalers: Food and Grocery,industry_name_Wholesalers: Health Care,Target
0,50000,Walmart,[Eduardo Brayner],0015I11484m7u87wBuD,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
1,50000,Costco,[Karim Krzan],0011S65285x2u62kQjM,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
2,50000,Target,[Michael McNamara],0018K30946c0u64tLmE,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
3,10580000,Exxon Mobil,[Ridwan Zurawik],0017K23751y0u72fZpZ,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,27
4,50000,Chevron,[David Seals],0010V22883q6u92eKgN,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,27


In [32]:
df=final_df_with_dummies.copy()
df.Amount1=pd.to_numeric(df.Amount1)
df.Amount1=((df.Amount1-df.Amount1.mean()).abs())/(df.Amount1.std())


for col in df.columns:
    if df[col].dtype=='object':
        df.drop(col,axis=1,inplace=True)

Y=df.Target
X=df.drop('Target',axis=1)
Y.head()

0    12
1    12
2    12
3    27
4    27
Name: Target, dtype: int64

In [33]:
df.head()

,Amount1,competitor_names_AIG,competitor_names_AT&T,competitor_names_Aetna,competitor_names_Albertsons Cos.,competitor_names_Alphabet,competitor_names_Amazon.com,competitor_names_American Airlines Group,competitor_names_AmerisourceBergen,competitor_names_Anthem,...,industry_name_Network and Other Communications Equipment,industry_name_Petroleum Refining,industry_name_Pharmaceuticals,industry_name_Pipelines,industry_name_Semiconductors and Other Electronic Components,industry_name_Specialty Retailers: Other,industry_name_Telecommunications,industry_name_Wholesalers: Food and Grocery,industry_name_Wholesalers: Health Care,Target
0,0.503836,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
1,0.503836,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
2,0.503836,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
3,0.339493,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,27
4,0.503836,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,27


In [34]:
X.head()

,Amount1,competitor_names_AIG,competitor_names_AT&T,competitor_names_Aetna,competitor_names_Albertsons Cos.,competitor_names_Alphabet,competitor_names_Amazon.com,competitor_names_American Airlines Group,competitor_names_AmerisourceBergen,competitor_names_Anthem,...,industry_name_Motor Vehicles and Parts,industry_name_Network and Other Communications Equipment,industry_name_Petroleum Refining,industry_name_Pharmaceuticals,industry_name_Pipelines,industry_name_Semiconductors and Other Electronic Components,industry_name_Specialty Retailers: Other,industry_name_Telecommunications,industry_name_Wholesalers: Food and Grocery,industry_name_Wholesalers: Health Care
0,0.503836,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.503836,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.503836,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.339493,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0.503836,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [35]:
def similarity_scorer(x,original_df,profile_id,threshold,pca=False,includeAmount=False): 
#     x=x.copy()
    profile_index=np.where(original_df['profile_id']==profile_id)
    profile_data=x.iloc[profile_index]
    
    if not includeAmount:
        x=x.drop('Amount1',axis=1)
        profile_data=profile_data.drop('Amount1',axis=1)
        
    if pca:
        pca = PCA(n_components=30)
        df2=pca.fit_transform(x)

        df2=pd.DataFrame(df2)
        sim=cosine_similarity(df2.iloc[profile_index],df2)
    else:
        sim=cosine_similarity(profile_data,x)

    indices=np.where(sim[0]>=threshold)

    similarity_scores={}
    for i in indices[0]:
        similarity_scores[original_df.iloc[i]['profile_id']]=sim[0][i]
    
    print(similarity_scores)
    return sorted(similarity_scores.items(), key=lambda kv: kv[1],reverse=True)



In [36]:
similarity_scorer(X,final_df,'0015I11484m7u87wBuD',0.5,True,False)

{'0015I11484m7u87wBuD': 0.9999999999999999, '0018K30946c0u64tLmE': 0.7756545909505524, '0019L27831f6u45aPfV': 0.5212273493230966}


[('0015I11484m7u87wBuD', 0.9999999999999999),
 ('0018K30946c0u64tLmE', 0.7756545909505524),
 ('0019L27831f6u45aPfV', 0.5212273493230966)]

In [37]:
final_df[final_df['profile_id']=='0015I11484m7u87wBuD']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
0,50000,Rosella Hymer,Amplify - QuickStart(Process Insights),Confirming Value with Power,Walmart,Costco,[Eduardo Brayner],General Merchandisers,0015I11484m7u87wBuD


In [38]:
final_df[final_df['profile_id']=='0018K30946c0u64tLmE']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
2,50000,Jayne Knox,Amplify - QuickStart(Process Insights),Confirming Value with Power,Target,Walmart,[Michael McNamara],General Merchandisers,0018K30946c0u64tLmE


In [39]:
final_df[final_df['profile_id']=='0019L27831f6u45aPfV']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
39,50000,Olly Breuninger,Amplify - QuickStart(Process Insights),Confirming Value with Power,Wells Fargo,Citigroup,[Mark Hammersmith],Commercial Banks,0019L27831f6u45aPfV


In [41]:
similarity_scorer(X,final_df,'0017O90702b2u22mRuO',0.5,True,False)

{'0017O90702b2u22mRuO': 0.9999999999999999, '0012S45318b8u92fVzB': 0.7670084398923686, '0018T72611b0u02hXaA': 0.5569700241383293, '0013B55715x9u43zCnH': 0.5111077458531279, '0014W44938l8u40sVqG': 0.5056667209907324}


[('0017O90702b2u22mRuO', 0.9999999999999999),
 ('0012S45318b8u92fVzB', 0.7670084398923686),
 ('0018T72611b0u02hXaA', 0.5569700241383293),
 ('0013B55715x9u43zCnH', 0.5111077458531279),
 ('0014W44938l8u40sVqG', 0.5056667209907324)]

In [42]:
final_df[final_df['profile_id']=='0017O90702b2u22mRuO']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
8,21483000,Janice Cerniglia,Cortex Enterprise(AI Develpoment),Negotiations,Berkshire Hathaway,AIG,[Courtney Bodey],Insurance: Property and Casualty (Stock),0017O90702b2u22mRuO


In [43]:
final_df[final_df['profile_id']=='0012S45318b8u92fVzB']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
45,19082000,Janice Cerniglia,Cortex Enterprise(AI Develpoment),Negotiations,Home Depot,Lowes,[Matt Carey],Specialty Retailers: Other,0012S45318b8u92fVzB


In [44]:
final_df[final_df['profile_id']=='0018T72611b0u02hXaA']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
53,40803000,Ridwan Zurawik,Cortex Enterprise(AI Develpoment),Negotiations,State Farm Insurance Cos.,Nationwide,[Lilly Raymo],Insurance: Property and Casualty (Mutual),0018T72611b0u02hXaA


In [46]:
# visualisation imports
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [47]:
def plot(X, Y, i, j, label1, label2):
    x_min, x_max = X[:, i].min() - .5, X[:, i].max() + .5
    y_min, y_max = X[:, j].min() - .5, X[:, j].max() + .5
    plt.figure(2, figsize=(8, 6))

    # clean the figure
    plt.clf()

    plt.scatter(X[:, i], X[:, j], c=Y, cmap=plt.cm.Paired)
    plt.xlabel(label1)
    plt.ylabel(label2)

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xticks(())
    plt.yticks(())

    plt.show()

In [60]:
from metric_learn import LMNN
lmnn = LMNN(k=1, learn_rate=1e-6)

lmnn.fit(X, Y)
X_lmnn = lmnn.transform()

X_lmnn
# plot(X_lmnn,Y,0,1,"Sepal Length", "Sepal Width")
# plot(X_lmnn,Y,2,3,"Petal Length", "Petal Width")
# plot(X_lmnn,Y,0,2,"Sepal Length", "Petal Length")
# plot(X_lmnn,Y,0,3,"Sepal Length", "Petal Width")
# plot(X_lmnn,Y,1,2,"Sepal Width", "Petal Length")
# plot(X_lmnn,Y,1,3,"Sepal Width", "Petal Width")


/Users/sswetank/anaconda3/lib/python3.6/site-packages/metric_learn/lmnn.py:62: UserWarning: use_pca does nothing for the python_LMNN implementation
  warnings.warn('use_pca does nothing for the python_LMNN implementation')


array([[-2.73630904e-03, -1.65069401e-02, -7.42873606e-03, ...,
        -5.04840541e-03, -3.20073827e-04,  1.04193783e-02],
       [-9.95112909e-02, -1.70539706e-02, -7.51222243e-03, ...,
        -3.72214239e-03, -8.04116403e-04, -4.87046352e-03],
       [-6.85085526e-03, -1.64554378e-02, -7.42873606e-03, ...,
         6.47440218e-03, -9.89143864e-04, -5.50036774e-03],
       ...,
       [-1.78371189e-01, -1.43517095e-01, -4.68886889e-02, ...,
        -6.46209714e-03, -1.23806392e-04,  2.48595608e-04],
       [ 3.03337278e-04, -6.43171056e-02, -7.42873606e-03, ...,
        -7.28874210e-03, -9.99561919e-04, -1.71719993e-03],
       [-9.86729548e-02, -1.66325720e-02, -7.51222243e-03, ...,
        -3.07442810e-03, -1.57055564e-03, -7.08962168e-03]])

In [63]:
X_lmnn=pd.DataFrame(X_lmnn)
similarity_scorer(X_lmnn,final_df,'0015I11484m7u87wBuD',0.5,True,True)

{'0015I11484m7u87wBuD': 1.0000000000000002, '0011S65285x2u62kQjM': 0.7248475544084197, '0018K30946c0u64tLmE': 0.8839677217542785}


[('0015I11484m7u87wBuD', 1.0000000000000002),
 ('0018K30946c0u64tLmE', 0.8839677217542785),
 ('0011S65285x2u62kQjM', 0.7248475544084197)]

In [64]:
final_df[final_df['profile_id']=='0015I11484m7u87wBuD']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
0,50000,Rosella Hymer,Amplify - QuickStart(Process Insights),Confirming Value with Power,Walmart,Costco,[Eduardo Brayner],General Merchandisers,0015I11484m7u87wBuD


In [65]:
final_df[final_df['profile_id']=='0018K30946c0u64tLmE']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
2,50000,Jayne Knox,Amplify - QuickStart(Process Insights),Confirming Value with Power,Target,Walmart,[Michael McNamara],General Merchandisers,0018K30946c0u64tLmE


In [66]:
final_df[final_df['profile_id']=='0011S65285x2u62kQjM']

,Amount1,ExecSponsor1,OpportunityName1,OpportunityStatus1,account_name,competitor_names,contacts,industry_name,profile_id
1,50000,Axel Whitsel,Engage - QuickStart(Coversational AI),Prospecting,Costco,Target,[Karim Krzan],General Merchandisers,0011S65285x2u62kQjM
